Load the dataset

In [37]:
import pandas as pd
file_path = "StudentsPerformance.csv"
data = pd.read_csv(file_path)
data.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Standardize column names

Check for Missing Values and Data Types

Check Descriptive Statistics for Score Columns

In [38]:
data.columns = data.columns.str.lower().str.replace(" ", "_")
data.isnull().sum()

,0
gender,0
race/ethnicity,0
parental_level_of_education,0
lunch,0
test_preparation_course,0
math_score,0
reading_score,0
writing_score,0


In [39]:
data.dtypes
score_columns_stats = data[['math_score', 'reading_score', 'writing_score']].describe()
score_columns_stats

,math_score,reading_score,writing_score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


Unique Values in Categorical Columns

In [40]:
unique_values = {col: data[col].unique() for col in data.select_dtypes(include='object').columns}
print("\nUnique Values in Categorical Columns:\n", unique_values)


Unique Values in Categorical Columns:
 {'gender': array(['female', 'male'], dtype=object), 'race/ethnicity': array(['group B', 'group C', 'group A', 'group D', 'group E'],
      dtype=object), 'parental_level_of_education': array(["bachelor's degree", 'some college', "master's degree",
       "associate's degree", 'high school', 'some high school'],
      dtype=object), 'lunch': array(['standard', 'free/reduced'], dtype=object), 'test_preparation_course': array(['none', 'completed'], dtype=object)}


 Identify Outliers

In [41]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

Check for outliers in each score column

In [42]:
math_outliers, math_lower, math_upper = detect_outliers_iqr(data, 'math_score')
reading_outliers, reading_lower, reading_upper = detect_outliers_iqr(data, 'reading_score')
writing_outliers, writing_lower, writing_upper = detect_outliers_iqr(data, 'writing_score')

print(f"\nMath Outliers (Lower Bound: {math_lower}, Upper Bound: {math_upper}):\n", math_outliers)
print(f"\nReading Outliers (Lower Bound: {reading_lower}, Upper Bound: {reading_upper}):\n", reading_outliers)
print(f"\nWriting Outliers (Lower Bound: {writing_lower}, Upper Bound: {writing_upper}):\n", writing_outliers)


Math Outliers (Lower Bound: 27.0, Upper Bound: 107.0):
      gender race/ethnicity parental_level_of_education         lunch  \
17   female        group B            some high school  free/reduced   
59   female        group C            some high school  free/reduced   
145  female        group C                some college  free/reduced   
338  female        group B            some high school  free/reduced   
466  female        group D          associate's degree  free/reduced   
787  female        group B                some college      standard   
842  female        group B                 high school  free/reduced   
980  female        group B                 high school  free/reduced   

    test_preparation_course  math_score  reading_score  writing_score  
17                     none          18             32             28  
59                     none           0             17             10  
145                    none          22             39             33  
338   

Handle Outliers - Option to Remove or Impute
Remove Outliers

Impute Outliers with Median

In [43]:
cleaned_data = data[(data['math_score'] >= math_lower) & (data['math_score'] <= math_upper) &
                    (data['reading_score'] >= reading_lower) & (data['reading_score'] <= reading_upper) &
                    (data['writing_score'] >= writing_lower) & (data['writing_score'] <= writing_upper)]
median_math = data['math_score'].median()
median_reading = data['reading_score'].median()
median_writing = data['writing_score'].median()

data['math_score'] = data['math_score'].apply(lambda x: median_math if x < math_lower or x > math_upper else x)
data['reading_score'] = data['reading_score'].apply(lambda x: median_reading if x < reading_lower or x > reading_upper else x)
data['writing_score'] = data['writing_score'].apply(lambda x: median_writing if x < writing_lower or x > writing_upper else x)



Display Cleaned Data

Save the cleaned data to a new CSV file

In [44]:
data.head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72.0,72.0,74.0
1,female,group C,some college,standard,completed,69.0,90.0,88.0
2,female,group B,master's degree,standard,none,90.0,95.0,93.0
3,male,group A,associate's degree,free/reduced,none,47.0,57.0,44.0
4,male,group C,some college,standard,none,76.0,78.0,75.0


In [45]:
data.to_csv("Cleaned_StudentsPerformance.csv", index=False)